<a href="https://colab.research.google.com/github/Manochithra/first_streamlit_app/blob/main/Sams_Copy_of_Building_RAG_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Building a RAG Application: A Comprehensive Demo**

In today's digital healthcare landscape, patients and medical professionals frequently seek accurate, up-to-date answers to a wide range of medical questions. However:

- Search engines return too much noise or unverified sources.

- Chatbots often hallucinate or give generic advice without referencing valid information.

- Medical professionals/Patients have limited time to find contextually relevant evidence or literature.

There is a growing need for an AI system that can:

- Provide precise, evidence-backed answers,

- Maintain context awareness during multi-turn conversations,

- And optionally enhance its knowledge using trusted real-time web sources.

### **Objective:**
In this notebook, you will explore the Retrieval-Augmented Generation (RAG) framework and its application to Large Language Models (LLMs), specifically in the context of a Medical Assistant Chatbot.

You will:

- **Set up the necessary libraries** and tools for implementing RAG with LLMs.

By the end of this demo, you will understand the RAG framework and its practical applications in enhancing LLMs.

\

---

\

### **What is RAG?**

**Retrieval-Augmented Generation (RAG)** is an advanced method to enhance **LLMs** with external knowledge. Instead of relying solely on the model's pre-trained knowledge, RAG allows the LLM to retrieve relevant information from external databases (like vector stores) during inference, making it suitable for dynamic and domain-specific applications.

RAG addresses many challenges faced by LLMs:
- **Domain Knowledge Gaps**: LLMs can be limited in specific knowledge areas, especially in evolving fields.
- **Factuality Issues & Hallucinations**: RAG helps reduce incorrect or fabricated answers by retrieving context from trusted data sources.
- **Real-Time Updates**: RAG enables the integration of continuously updated external knowledge without retraining the model.

\

---

\


### **Why Use RAG?**

RAG enhances the capabilities of LLMs by:
- **Improving Response Quality**: Provides context from up-to-date knowledge, making the response more accurate.
- **Handling Knowledge Gaps**: Allows LLMs to access external databases, which are often required for specialized tasks.
- **Reducing Hallucinations**: RAG reduces the risk of LLMs making up answers or providing misleading responses.
- **Faster Deployment**: RAG avoids the need for retraining the model for every new dataset or domain.

> For more detailed information, you can refer to this [RAG Paper](https://arxiv.org/abs/2312.10997) or the [Retrieval-Augmented Generation Blog](https://www.promptingguide.ai/research/rag.en#introduction-to-rag).



### **Step 1. Setup and Install Dependencies**

> Run the following cell to install dependencies required:

In [ ]:
# Import the warnings module and suppress any warnings that might appear during execution
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings to keep the output clean and focused

- The `warnings` module is used to manage and control the warning messages in Python.
- `filterwarnings("ignore")` tells Python to ignore all warning messages that may be raised during execution. This is often done to keep the notebook or script output cleaner, especially when warnings are known and not critical to the code's functionality.

In [ ]:
!pip install -q gradio
!pip install -q openai
!pip install -q faiss-cpu==1.8.0
!pip install -q langchain
!pip install -q langchain-classic
!pip install -q langchain-openai
!pip install -q langchain-community
!pip install -q langchain-core
!pip install -q langchain-text-splitters
#!pip install -q langchain-huggingface
#!pip install -q transformers
!pip install -q --upgrade gradio gradio_client
#!pip install -q "websockets>=15.0.1"


In [ ]:
!pip install -q --upgrade websockets

### **Step 2. Imports and Configuration**

In [ ]:
import os
import openai
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

### **Step 3. Add OpenAI API Key**

[Models available](https://platform.openai.com/docs/models)

#### **📌 Best Practices for API Key Safety**


1. Always use a unique API key for each team member on your account
2. Never deploy your key in client-side environments like browsers or mobile apps
3. Never commit your key to your repository
4. Use Environment Variables in place of your API key
5. Use a Key Management Service
6. Monitor your account usage and rotate your keys when needed



In [ ]:
from google.colab import userdata

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

#### **🛡️ Further Details**

1. Use .env Files for Local Development
  - Store the API key in a .env file and load it using dotenv.

    ```
    from dotenv import load_dotenv
    load_dotenv()
    API_KEY = os.getenv("OPENAI_API_KEY")
    ```
2. Use Secret Managers for Production
  - AWS: AWS Secrets Manager
  - Azure: Azure Key Vault
  - GCP: Google Secret Manager

3. Use Token-Based Authentication if Available
  - Some cloud services provide temporary tokens instead of static API keys.

### **Step 4. Text Cleaning and Preprocessing**

> We now clean and preprocess the data. The text needs to be formatted before it can be processed into vector embeddings.

#### **1. Upload the Dataset**
> (dataset.txt)

In [ ]:
from google.colab import files

# Upload the CSV files
uploaded = files.upload()

# Verify files uploaded successfully
print(uploaded)

Saving ai-medical-chatbot.txt to ai-medical-chatbot (1).txt
{'ai-medical-chatbot (1).txt': b"# About Medical Conditions\n\n### What are Medical Conditions?\n\nMedical conditions are health problems or diseases that affect the normal functioning of the human body. They can be acute (short-term) or chronic (long-term) and may require medical attention, ongoing treatment, or lifestyle modifications. Understanding medical conditions is essential for proper healthcare management and early intervention.\n\n### Who needs Medical Consultation?\n\nAll individuals experiencing symptoms, having risk factors, or requiring preventive care should seek medical consultation to maintain optimal health and detect conditions early.\n\n### Types of Medical Conditions\n\nIn healthcare, several types of medical conditions exist: Acute conditions and Chronic conditions.\n\n1. Acute Medical Conditions: Acute conditions develop suddenly and require immediate medical attention. They include conditions like hear

In [ ]:
# Read and clean the dataset
with open('ai-medical-chatbot.txt', 'r') as f:
    data = f.read()

data = data.replace('\n\n','\n') # Clean unnecessary line breaks
data = data.split('---') # Split the data into separate sections based on a delimiter

In [ ]:
len(data)

5

In [ ]:
print(data[3])


# Medical History Assessment
### **What is Medical History Assessment?**
Medical History Assessment is a comprehensive evaluation process used to collect detailed information about a patient's past and current health conditions, family medical history, medications, and lifestyle factors for accurate diagnosis and treatment planning.
### **Information Required for Medical History Assessment**
Here are the essential details that should be provided during Medical History Assessment:
1. Current symptoms and their duration
2. Past medical conditions and treatments
3. Current medications and supplements
4. Family medical history
5. Allergies and adverse reactions
6. Surgical history
7. Social history (smoking, alcohol, exercise)
8. Reproductive history (for women)
9. Immunization records



In [ ]:
print(data[4])


# FAQs about Medical Conditions
**Is it necessary to see a doctor for minor symptoms?**
It depends on the symptoms and their persistence. However, it's better to consult a healthcare provider for proper evaluation, especially if symptoms persist or worsen, rather than self-diagnosing.
**Are chronic conditions different from acute conditions?**
Yes, chronic conditions are long-term health problems requiring ongoing management, while acute conditions develop suddenly and may resolve with appropriate treatment. The approach to treatment and management differs significantly between the two.
**Can I get medical consultation from another state?**
Yes, you can receive medical consultation from healthcare providers in another state through telemedicine services. However, some states have specific licensing requirements for out-of-state medical practice.
**Is medical consultation mandatory for prescription medications?**
Yes, prescription medications require a valid prescription from a license

In [ ]:
# Preprocess the text (adjust formatting for clean text)
for i in range(0, len(data)):
    if i==4:
      data[i] = data[i].replace('\n**', '\n###').replace('**','')
    else:
      data[i] = data[i].replace('**','')

In [ ]:
print(data[3])


# Medical History Assessment
### What is Medical History Assessment?
Medical History Assessment is a comprehensive evaluation process used to collect detailed information about a patient's past and current health conditions, family medical history, medications, and lifestyle factors for accurate diagnosis and treatment planning.
### Information Required for Medical History Assessment
Here are the essential details that should be provided during Medical History Assessment:
1. Current symptoms and their duration
2. Past medical conditions and treatments
3. Current medications and supplements
4. Family medical history
5. Allergies and adverse reactions
6. Surgical history
7. Social history (smoking, alcohol, exercise)
8. Reproductive history (for women)
9. Immunization records



In [ ]:
print(data[4])


# FAQs about Medical Conditions
###Is it necessary to see a doctor for minor symptoms?
It depends on the symptoms and their persistence. However, it's better to consult a healthcare provider for proper evaluation, especially if symptoms persist or worsen, rather than self-diagnosing.
###Are chronic conditions different from acute conditions?
Yes, chronic conditions are long-term health problems requiring ongoing management, while acute conditions develop suddenly and may resolve with appropriate treatment. The approach to treatment and management differs significantly between the two.
###Can I get medical consultation from another state?
Yes, you can receive medical consultation from healthcare providers in another state through telemedicine services. However, some states have specific licensing requirements for out-of-state medical practice.
###Is medical consultation mandatory for prescription medications?
Yes, prescription medications require a valid prescription from a licensed he

In [ ]:
# Organize the data into question-answer pairs
ques_ans = dict()
for i in range(0, len(data)):
    topics = data[i].split('\n###')
    for topic in topics[1:]:
      question_answer_pair = topic.split('\n')
      ques_ans[question_answer_pair[0]] = " ".join(question_answer_pair[1:])

In [ ]:
len(ques_ans)

48

In [ ]:
ques_ans.keys()

dict_keys([' What are Medical Conditions?', ' Who needs Medical Consultation?', ' Types of Medical Conditions', ' Why do patients need early diagnosis?', ' Importance of Medical Consultation for different health conditions', ' How can patients seek new Medical Consultation', ' Documents required for new Medical Consultation', ' Cost of new Medical Consultation', ' Time required to get Medical Consultation', ' Information that can be updated in Medical Records', ' General process to update Medical Records', ' Documents required to update Medical Records', ' Documents required to update emergency contacts', ' Process to change primary care physician', ' Obtaining Medical Records', ' Documents required for obtaining Medical Records', ' Charges for Medical Records', ' Medical Test Coordination', ' Process for Medical Test Coordination', ' HealthCare fees for test coordination', ' Documents required for Medical Test Coordination', ' Time required for Medical Test coordination?', ' What is M

In [ ]:
print(ques_ans[" What are Medical Conditions?"])

Medical conditions are health problems or diseases that affect the normal functioning of the human body. They can be acute (short-term) or chronic (long-term) and may require medical attention, ongoing treatment, or lifestyle modifications. Understanding medical conditions is essential for proper healthcare management and early intervention.


In [ ]:
all_content = str()
for key, value in ques_ans.items():
    # print(key)
    all_content += key + " " + value + "\n"
# print(all_content)
# Print cleaned content for verification

In [ ]:
print(all_content)

 What are Medical Conditions? Medical conditions are health problems or diseases that affect the normal functioning of the human body. They can be acute (short-term) or chronic (long-term) and may require medical attention, ongoing treatment, or lifestyle modifications. Understanding medical conditions is essential for proper healthcare management and early intervention.
 Who needs Medical Consultation? All individuals experiencing symptoms, having risk factors, or requiring preventive care should seek medical consultation to maintain optimal health and detect conditions early.
 Types of Medical Conditions In healthcare, several types of medical conditions exist: Acute conditions and Chronic conditions. 1. Acute Medical Conditions: Acute conditions develop suddenly and require immediate medical attention. They include conditions like heart attacks, strokes, infections, and injuries that need prompt treatment to prevent complications. 2. Chronic Medical Conditions: Chronic conditions ar

#### **2. Text Chunking**

> To prepare the text for semantic search, we break the content into smaller chunks. This will make it easier to process and index efficiently.


In [ ]:
# Split the large content into smaller chunks for indexing
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=300, chunk_overlap=128, length_function=len)

chunks = text_splitter.split_text(all_content)

# Display the first chunk for verification
print(chunks)

['What are Medical Conditions? Medical conditions are health problems or diseases that affect the normal functioning of the human body. They can be acute (short-term) or chronic (long-term) and may require medical attention, ongoing treatment, or lifestyle modifications. Understanding medical conditions is essential for proper healthcare management and early intervention.', 'Who needs Medical Consultation? All individuals experiencing symptoms, having risk factors, or requiring preventive care should seek medical consultation to maintain optimal health and detect conditions early.', 'Types of Medical Conditions In healthcare, several types of medical conditions exist: Acute conditions and Chronic conditions. 1. Acute Medical Conditions: Acute conditions develop suddenly and require immediate medical attention. They include conditions like heart attacks, strokes, infections, and injuries that need prompt treatment to prevent complications. 2. Chronic Medical Conditions: Chronic conditio

In [ ]:
len(chunks)

47

#### **4. Store Embeddings in FAISS**

> We use FAISS to store and index the vectors. FAISS is optimized for similarity search, allowing us to retrieve the most relevant vectors during the search phase.

In [ ]:
embeddings = OpenAIEmbeddings()

vectorStore = FAISS.from_texts(chunks, embeddings)
vectorStore.save_local("faiss_doc_idx")

#### **5. Perform Semantic Search**

> Now that we have indexed the embeddings, we can perform a semantic search. This search will find the most relevant documents based on a query, considering the meaning rather than exact matches.

In [ ]:
docs = vectorStore.similarity_search("What are the main differences between acute and chronic medical conditions?")

"""
- What does abutment of the nerve root mean in MRI reports?
- What is the treatment for L5-S1 disc bulge?
- What does mild concentric left ventricular hypertrophy mean?
- How can I lose weight with hypothyroidism?
"""

'\n- What does abutment of the nerve root mean in MRI reports?\n- What is the treatment for L5-S1 disc bulge?\n- What does mild concentric left ventricular hypertrophy mean?\n- How can I lose weight with hypothyroidism?\n'

In [ ]:
len(docs)

4

In [ ]:
for doc in docs:
    print(doc.page_content)

Are chronic conditions different from acute conditions? Yes, chronic conditions are long-term health problems requiring ongoing management, while acute conditions develop suddenly and may resolve with appropriate treatment. The approach to treatment and management differs significantly between the two.
Types of Medical Conditions In healthcare, several types of medical conditions exist: Acute conditions and Chronic conditions. 1. Acute Medical Conditions: Acute conditions develop suddenly and require immediate medical attention. They include conditions like heart attacks, strokes, infections, and injuries that need prompt treatment to prevent complications. 2. Chronic Medical Conditions: Chronic conditions are long-term health problems that persist over time and require ongoing management. They include diabetes, hypertension, arthritis, and other conditions that need continuous care and monitoring. Both acute and chronic conditions require proper medical evaluation and treatment. The a

#### 6. **Setup OpenAI Model**


In [ ]:
from langchain_community.llms import HuggingFaceHub

In [ ]:
import os
from langchain_classic.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA
from langchain_classic.llms import OpenAI

In [ ]:
question = "What are the main differences between acute and chronic medical conditions?"


llm = OpenAI()
template = """You are a medical assistant chatbot helping answer patient questions based only on the provided context.
Do not guess or provide inaccurate information. If the answer is not found in the context, say you don’t know.
You will answer the question based on the context - {context}.
Question: {question}
Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Create Retrieval QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorStore.as_retriever(search_type="similarity", k=4),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

# Run the chain with the user query
result = qa_chain({"query": question})
answer = result['result']

In [ ]:
print("answer is\n", answer)

answer is
  The main differences between acute and chronic medical conditions are the time of onset, duration, and approach to treatment and management. Acute conditions develop suddenly and require immediate medical attention, while chronic conditions are long-term health problems that require ongoing management.


In [ ]:
print("answer is\n", answer)

answer is
  The main differences between acute and chronic medical conditions are the time of onset, duration, and approach to treatment and management. Acute conditions develop suddenly and require immediate medical attention, while chronic conditions are long-term health problems that require ongoing management.


In [ ]:
result.keys()

dict_keys(['query', 'result'])

### **Step 5. Gradio Chatbot**

In [ ]:
from langchain_classic.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA

In [ ]:
from langchain_classic.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA
from langchain_classic.chat_models import ChatOpenAI
from langchain_classic.schema import AIMessage, HumanMessage
import gradio as gr
import traceback

# Toggle this flag to True to enable debug logs
DEBUG = True

def predict(message, history):
    try:
        if DEBUG: print(f"\n📥 Received message: {message}")
        history_langchain_format = []

        # Convert chat history into LangChain format
        for human, ai in history:
            history_langchain_format.append(HumanMessage(content=human))
            history_langchain_format.append(AIMessage(content=ai))

        # Define Prompt
        template = """You are a medical assistant chatbot helping answer patient questions based only on the provided context.
        Do not guess or provide inaccurate information. If the answer is not found in the context, say you don’t know.
        You will answer the question based on the context - {context}.
        Question: {question}
        Answer:"""
        QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

        # Create Retrieval QA Chain
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=vectorStore.as_retriever(search_type="similarity", k=4),
            chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
        )

        # Run the chain with the user query
        result = qa_chain({"query": message})
        answer = result['result']

        if DEBUG: print(f"✅ Retrieved Answer:\n{answer}")

        # Update history
        history_langchain_format.append(HumanMessage(content=message))
        history_langchain_format.append(AIMessage(content=answer))

        return answer

    except Exception as e:
        error_trace = traceback.format_exc()
        if DEBUG: print(f"❌ Exception occurred:\n{error_trace}")
        return "⚠️ Sorry, an internal error occurred. Please try again later."

## Gradio Chatbot UI
# gr.ChatInterface(
#     fn=predict,
#     chatbot=gr.Chatbot(height=300),
#     textbox=gr.Textbox(placeholder="Ask me a question related to Healthcare and Medical Services", container=False, scale=7),
#     title="DocumentQABot",
#     examples=[
#         "What are the main differences between acute and chronic medical conditions?",
#         "What are the main differences between acute and chronic medical conditions?",
#         "What does mild concentric LV hypertrophy mean?"
#     ],
# ).launch(share=True)

In [ ]:
import gradio as gr
import traceback

custom_css = """
@import url('https://fonts.googleapis.com/css2?family=DM+Sans:ital,opsz,wght@0,9..40,300;0,9..40,400;0,9..40,500;0,9..40,600;0,9..40,700&display=swap');

* { font-family: 'DM Sans', sans-serif !important; }

.gradio-container {
    max-width: 800px !important;
    margin: 0 auto !important;
    background: #f0f5ff !important;
}

.app-header {
    background: linear-gradient(135deg, #1e40af 0%, #1d4ed8 60%, #2563eb 100%);
    border-radius: 16px;
    padding: 28px 32px;
    margin-bottom: 16px;
    text-align: center;
}
.app-header h1 {
    font-size: 24px;
    font-weight: 700;
    color: white !important;
    margin: 0 0 6px 0;
}
.app-header p {
    font-size: 13px;
    color: rgba(255,255,255,0.8) !important;
    margin: 0;
}

.chat-wrap {
    background: white;
    border-radius: 16px;
    padding: 8px;
    box-shadow: 0 2px 16px rgba(30,64,175,0.08);
    margin-bottom: 12px;
}

.input-wrap {
    background: white;
    border-radius: 16px;
    padding: 14px 16px;
    box-shadow: 0 2px 16px rgba(30,64,175,0.08);
    display: flex;
    gap: 10px;
    align-items: flex-end;
}

textarea {
    border-radius: 10px !important;
    border: 1.5px solid #dbeafe !important;
    font-size: 14px !important;
    padding: 10px 14px !important;
    resize: none !important;
    transition: border 0.2s !important;
}
textarea:focus {
    border-color: #1d4ed8 !important;
    box-shadow: 0 0 0 3px rgba(29,78,216,0.1) !important;
    outline: none !important;
}

button.primary {
    background: linear-gradient(135deg, #1e40af, #2563eb) !important;
    border-radius: 10px !important;
    font-weight: 600 !important;
    font-size: 14px !important;
    color: white !important;
    border: none !important;
    padding: 10px 22px !important;
    transition: opacity 0.2s !important;
}
button.primary:hover { opacity: 0.88 !important; }

.badge-row {
    display: flex;
    justify-content: center;
    gap: 8px;
    margin-top: 14px;
    flex-wrap: wrap;
}
.badge {
    background: white;
    border-radius: 20px;
    padding: 5px 14px;
    font-size: 12px;
    color: #475569;
    box-shadow: 0 1px 6px rgba(0,0,0,0.07);
    border: 1px solid #e2e8f0;
}
"""

DEBUG = True

def respond(message, history):
    try:
        if DEBUG: print(f"\n📥 {message}")

        # Convert Gradio history → LangChain format
        lc_history = []
        for turn in history:
            lc_history.append(HumanMessage(content=turn["content"] if isinstance(turn, dict) else turn[0]))
            if isinstance(turn, dict):
                pass
            else:
                lc_history.append(AIMessage(content=turn[1]))

        template = """You are a medical assistant chatbot helping answer patient questions based only on the provided context.
Do not guess or provide inaccurate information. If the answer is not found in the context, say you don't know.
Context: {context}
Question: {question}
Answer:"""
        QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=vectorStore.as_retriever(search_type="similarity", k=4),
            chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
        )
        result = qa_chain({"query": message})
        answer = result["result"]

        if DEBUG: print(f"✅ {answer[:100]}...")
        return answer

    except Exception:
        if DEBUG: print(traceback.format_exc())
        return "⚠️ Sorry, something went wrong. Please try again."


with gr.Blocks(css=custom_css, title="Medical Assistant") as demo:

    gr.HTML("""
        <div class="app-header">
            <h1>🏥 Medical Assistant</h1>
            <p>RAG-powered · Evidence-backed · Built on verified medical knowledge</p>
        </div>
    """)

    with gr.Column(elem_classes="chat-wrap"):
        chatbot = gr.Chatbot(
            height=420,
            layout="bubble",
            placeholder="<strong>Ask me anything about medical conditions or healthcare services</strong>",
            avatar_images=(None, "https://api.dicebear.com/7.x/bottts-neutral/svg?seed=medbot"),
            show_label=False,
        )

    with gr.Row(elem_classes="input-wrap"):
        txt = gr.Textbox(
            placeholder="e.g. What are the differences between acute and chronic conditions?",
            container=False,
            scale=9,
            lines=1,
            max_lines=4,
        )
        send = gr.Button("Send →", variant="primary", scale=1)

    gr.Examples(
        examples=[
            "What are the main differences between acute and chronic medical conditions?",
            "How can I prepare for a medical consultation?",
            "Can I see multiple specialists for different conditions?",
            "What does mild concentric LV hypertrophy mean?",
        ],
        inputs=txt,
        label="✨ Try an example",
    )

    gr.HTML("""
        <div class="badge-row">
            <span class="badge">🔒 Evidence-backed</span>
            <span class="badge">⚕️ Medical knowledge base</span>
            <span class="badge">🤖 GPT-3.5 powered</span>
            <span class="badge">⚡ FAISS retrieval</span>
        </div>
    """)

    history_state = gr.State([])

    def handle(message, history):
        answer = respond(message, history)
        history = history + [{"role": "user", "content": message},
                             {"role": "assistant", "content": answer}]
        return "", history, history

    txt.submit(handle, [txt, history_state], [txt, chatbot, history_state])
    send.click(handle, [txt, history_state], [txt, chatbot, history_state])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a600fd3909274723a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

# Dummy function (replace with your actual LLM logic)
def predict(message, history):
    return f"This is a placeholder response for your medical query regarding: {message}"

# 1. Define a clean, light theme using Gradio's native theme builder
custom_theme = gr.themes.Soft(
    primary_hue="sky",
    secondary_hue="blue",
    neutral_hue="slate",
    font=[gr.themes.GoogleFont("DM Sans"), "sans-serif"]
).set(
    body_background_fill="#F9FAFB", # Very light grey background
    block_background_fill="#FFFFFF", # White blocks for contrast
    block_border_width="1px",
    block_border_color="#E5E7EB",
    block_shadow="0px 4px 12px rgba(0, 0, 0, 0.05)",
    button_primary_background_fill="#0EA5E9",
    button_primary_text_color="#FFFFFF",
)

# 2. Inject custom CSS to force the font everywhere and style the chat bubbles
custom_css = """
/* Import DM Sans */
@import url('https://fonts.googleapis.com/css2?family=DM+Sans:opsz,wght@9..40,400;9..40,500;9..40,600&display=swap');

/* Force font application */
* { font-family: 'DM Sans', sans-serif !important; }

/* Restrict maximum width for better readability on large screens */
.gradio-container { max-width: 850px !important; margin: auto; }

/* Style the Bot's message bubble */
div.message.bot {
    background-color: #FFFFFF;
    border: 1px solid #E5E7EB;
    border-radius: 8px;
    box-shadow: 0 1px 2px rgba(0,0,0,0.05);
}

/* Style the User's message bubble */
div.message.user {
    background-color: #E0F2FE; /* Light sky blue */
    border: 1px solid #BAE6FD;
    border-radius: 8px;
}
"""

# 3. Wrap the ChatInterface in a gr.Blocks context to apply the theme and CSS
with gr.Blocks(theme=custom_theme, css=custom_css) as demo:

    # Add a custom header instead of the default ChatInterface title
    gr.Markdown("<h1 style='text-align: center; color: #1E293B; margin-top: 20px;'>🩺 DocumentQABot</h1>")
    gr.Markdown("<p style='text-align: center; color: #64748B; font-size: 16px; margin-bottom: 20px;'>Your intelligent healthcare and medical services assistant.</p>")

    gr.ChatInterface(
        fn=predict,
        chatbot=gr.Chatbot(
            height=500, # Increased height so it doesn't feel cramped
            show_label=False,
            # Add custom avatars (using free DiceBear SVGs for the demo)
            avatar_images=(
                "https://api.dicebear.com/7.x/notionists/svg?seed=User",
                "https://api.dicebear.com/7.x/bottts/svg?seed=MedicBot"
            ),
        ),
        textbox=gr.Textbox(
            placeholder="Ask me a question related to Healthcare and Medical Services...",
            container=False,
            scale=7
        ),
        examples=[
            "What are the main differences between acute and chronic medical conditions?",
            "What are the most common symptoms of seasonal allergies?",
            "What does mild concentric LV hypertrophy mean?"
        ],
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aa1bcd89c1566e718d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
